In [1]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import math, random
from datetime import datetime
import datetime as dt
import yfinance as yf
import preprocessor as p
import re
from sklearn.linear_model import LinearRegression
from textblob import TextBlob
import nltk
import requests
from keras.models import load_model
import os

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [7]:
def construct_lstm_model(df, sentiment_score):
    dataset_train=df.iloc[0:int(0.8*len(df)),:]
    dataset_test=df.iloc[int(0.8*len(df)):,:]
    
    training_set=df.iloc[:,4:5].values

    from sklearn.preprocessing import MinMaxScaler
    sc=MinMaxScaler(feature_range=(0,1))
    training_set_scaled=sc.fit_transform(training_set)
    
    X_train=[]
    y_train=[]
    for i in range(7,len(training_set_scaled)):
        X_train.append(np.append(training_set_scaled[i-7:i,0], sentiment_score))
        y_train.append(training_set_scaled[i,0])
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    X_forecast=np.array(X_train[-1,1:])
    X_forecast=np.append(X_forecast,y_train[-1])
    X_train=np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
    X_forecast=np.reshape(X_forecast, (1,X_forecast.shape[0],1))
    
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.layers import LSTM
    
    real_stock_price=dataset_test.iloc[:,4:5].values
    
    dataset_total=pd.concat((dataset_train['Close'],dataset_test['Close']),axis=0) 
    testing_set=dataset_total[ len(dataset_total) -len(dataset_test) -7: ].values
    testing_set=testing_set.reshape(-1,1)
    
    testing_set=sc.transform(testing_set)
    
    X_test=[]
    for i in range(7,len(testing_set)):
        X_test.append(testing_set[i-7:i,0])
    X_test=np.array(X_test)


    X_test=np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    regressor = load_model("lstm_model.weights.h5")
    
    predicted_stock_price=regressor.predict(X_test)
    
    error_lstm = math.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
    
    
    forecasted_stock_price=regressor.predict(X_forecast)
    
    forecasted_stock_price=sc.inverse_transform(forecasted_stock_price)
    
    lstm_pred=forecasted_stock_price[0,0]
    return lstm_pred,error_lstm
     


In [9]:
def train_lstm_model(df, sentiment_score):
    dataset_train=df.iloc[0:int(0.8*len(df)),:]
    dataset_test=df.iloc[int(0.8*len(df)):,:]
    
    training_set=df.iloc[:,4:5].values

    from sklearn.preprocessing import MinMaxScaler
    sc=MinMaxScaler(feature_range=(0,1))
    training_set_scaled=sc.fit_transform(training_set)
    
    X_train=[]
    y_train=[]
    for i in range(7,len(training_set_scaled)):
        X_train.append(np.append(training_set_scaled[i-7:i,0], sentiment_score))
        y_train.append(training_set_scaled[i,0])
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    X_forecast=np.array(X_train[-1,1:])
    X_forecast=np.append(X_forecast,y_train[-1])
    X_train=np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
    X_forecast=np.reshape(X_forecast, (1,X_forecast.shape[0],1))
    
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.layers import Dropout
    from keras.layers import LSTM

    if os.path.exists("lstm_model.weights.h5"):
        print("Loading existing model weights...")
        regressor = load_model("lstm_model.weights.h5")
        regressor.compile(optimizer='adam',loss='mean_squared_error')
        regressor.fit(X_train,y_train,epochs=25,batch_size=32 )
    
        regressor.save("lstm_model.weights.h5")
    else:
        print("Weights file not found. Creating and training a new model...")
        regressor=Sequential()
        
        regressor.add(LSTM(units=50,return_sequences=True,input_shape=(X_train.shape[1],1)))
        regressor.add(Dropout(0.1))
        
        regressor.add(LSTM(units=50,return_sequences=True))
        regressor.add(Dropout(0.1))
        
        regressor.add(LSTM(units=50,return_sequences=True))
        regressor.add(Dropout(0.1))
        
        regressor.add(LSTM(units=50))
        regressor.add(Dropout(0.1))
        
        regressor.add(Dense(units=1))
        
        regressor.compile(optimizer='adam',loss='mean_squared_error')

        regressor.fit(X_train,y_train,epochs=25,batch_size=32 )
    
    regressor.save("lstm_model.weights.h5")
    evaluation_metrics = evaluate_model(regressor, sc, dataset_test, df.copy(), sentiment_score)
    print("model metrics", evaluation_metrics)

In [11]:
def get_historical(quote):
    end = datetime.now()
    start = datetime(end.year-2,end.month,end.day)
    data = yf.download(quote, start=start, end=end)
    df = pd.DataFrame(data=data)
    df.to_csv(''+quote+'.csv')
    return

In [13]:
def get_sentiment_score(ticker):
    sia = SentimentIntensityAnalyzer()
    news_api_url = f'https://newsapi.org/v2/everything?q={ticker}&apiKey=9ce9c2ef3b6d46c7ba21948ae221a2d5'
    response = requests.get(news_api_url)
    
    if response.status_code == 200:
        articles = response.json().get('articles', [])
        scores = [sia.polarity_scores(article['title'])['compound'] for article in articles if 'title' in article]
        return np.mean(scores) if scores else 0
    return 0

In [15]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from pandas import json_normalize
import requests
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

def analyze_sentiment(text):
    scores = sia.polarity_scores(text)
    return pd.Series([scores['compound'], scores['neg'], scores['pos'], scores['neu']])

API_KEY = "MO80S3QD8B8COBBV"

def get_news_sentiment(ticker):
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&limit=1000&apikey={API_KEY}"
    response = requests.get(url).json()
    news_items = response.get("feed", [])
    df = pd.DataFrame(news_items)
    # df = pd.read_json("TSLA_News.json")
    df["time_published"] = pd.to_datetime(df["time_published"], format="%Y%m%dT%H%M%S")
    df["Date"] = df["time_published"].dt.strftime("%Y-%m-%d")
    df[['compound', 'neg', 'pos', 'neu']] = df["summary"].apply(analyze_sentiment)
    df_grouped = df.groupby('Date')[['compound', 'neg', 'pos', 'neu']].mean().reset_index()
    print("News grouped df", df_grouped)
    return df_grouped, df['title'].head(3)

def fetch_reddit_sentiment(query):
    url = f"https://www.reddit.com/search.json?q={query}&t=day"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers).json()
    df = json_normalize(response['data']['children'])
    df['Date'] = pd.to_datetime(df['data.created_utc'], unit='s').dt.strftime("%Y-%m-%d")
    df[['compound', 'neg', 'pos', 'neu']] = df["data.title"].apply(analyze_sentiment)
    df_grouped = df.groupby('Date')[['compound', 'neg', 'pos', 'neu']].mean().reset_index()
    print("Reddit grouped df\n")
    print(df_grouped)
    return df_grouped

In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
def evaluate_model(regressor, sc, dataset_test, df, sentiment_score):

    real_stock_price = df.iloc[int(0.8 * len(df)):, 4:5].values
    dataset_total = df.iloc[:, 4:5]
    inputs = dataset_total[len(dataset_total) - len(dataset_test) - 7:].values
    inputs = sc.transform(inputs)

    X_test = []
    for i in range(7, len(inputs)):
        X_test.append(np.append(inputs[i-7:i, 0], sentiment_score))

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = regressor.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)

    real_stock_price = real_stock_price.astype(float)
    predicted_stock_price = predicted_stock_price.astype(float)

    rmse = np.sqrt(mean_squared_error(real_stock_price, predicted_stock_price))
    mae = mean_absolute_error(real_stock_price, predicted_stock_price)
    mse = mean_squared_error(real_stock_price, predicted_stock_price)
    mape = np.mean(np.abs((real_stock_price - predicted_stock_price) / real_stock_price)) * 100


    return {
        "RMSE": rmse,
        "MAE": mae,
        "MSE": mse,
        "MAPE": mape
    }

In [ ]:
from flask import Flask, render_template_string, request, jsonify
import yfinance as yf
import random

app = Flask(__name__)

STOCK_SYMBOLS = ["AAPL", "GOOGL", "AMZN", "MSFT", "TSLA", "NFLX", "NVDA", "META", "BABA", "INTC"]

def fetch_real_time_stock_data(symbol, days):
    stock = yf.Ticker(symbol)
    hist = stock.history(period=f"{days}d")
    if hist.empty:
        return None
    
    stock_data = {
        "latest": hist['Close'][-1],
        "date": hist.index[-1].strftime('%Y-%m-%d'),
        "price": hist['Close'].tolist(),
        "labels": hist.index.strftime('%Y-%m-%d').tolist() 
    }
    return stock_data

def generate_sentiment_data(symbol, days):
    ticker = symbol
    news_sentiment, headlines = get_news_sentiment(ticker)
    reddit_keywords = {"AAPL":"apple", "GOOGL":"google", "AMZN":"amazon", "MSFT":"microsoft", "TSLA":"tesla", "NFLX":"netflix", "NVDA":"nvidia", "META":"meta", "BABA":"alibaba", "INTC":"intel"}
    reddit_sentiment = fetch_reddit_sentiment(reddit_keywords[ticker])
    
    mean_sentiment = pd.concat([news_sentiment, reddit_sentiment]).groupby('Date').mean().reset_index().sort_values(by='Date', ascending=True).head(days)
    print('mean sentiment', mean_sentiment)

    return {
        "labels": ["Positive", "Neutral", "Negative"],
        "values": [mean_sentiment['pos'].mean(), mean_sentiment['neu'].mean(), mean_sentiment['neg'].mean()],
        "news": [
            headlines[0],
            headlines[1],
            headlines[2]
        ]
    }

@app.route('/stock-data', methods=['GET'])
def stock_data():
    symbol = request.args.get('symbol', 'AAPL').upper()
    days = int(request.args.get('days', 30))
    stock_data = fetch_real_time_stock_data(symbol, days)
    if stock_data:
        return jsonify(stock_data)
    return jsonify({"error": "Unable to fetch stock data."})

@app.route('/predict-stock-value', methods=['GET'])
def fetch_model_prediction():
    symbol = request.args.get('symbol', 'AAPL').upper()
    print("SYMBOL", symbol)
    lstm_pred = getPredictions(symbol)
    return jsonify({
        "value": str(lstm_pred)
    })

@app.route('/start-background-training', methods=['GET'])
def start_background_training():
    symbol = request.args.get('symbol', 'AAPL').upper()
    print("SYMBOL", symbol)
    start_async_training(symbol)
    return jsonify({})

def getPredictions(quote):
    get_historical(quote)
    df = pd.read_csv(''+quote+'.csv')
    today_stock=df.iloc[-1:]
    code_list=[]
    for i in range(0,len(df)):
        code_list.append(quote)
    df2=pd.DataFrame(code_list,columns=['Code'])
    df2 = pd.concat([df2, df], axis=1)
    df=df2
    df = df[2:]
    df = df[:-1]
    sentiment_score = get_sentiment_score(quote)
    lstm_pred, error_lstm=construct_lstm_model(df, sentiment_score)
    return lstm_pred

def start_async_training(quote):
    get_historical(quote)
    df = pd.read_csv(''+quote+'.csv')
    today_stock=df.iloc[-1:]
    code_list=[]
    for i in range(0,len(df)):
        code_list.append(quote)
    df2=pd.DataFrame(code_list,columns=['Code'])
    df2 = pd.concat([df2, df], axis=1)
    df=df2
    df = df[2:]
    df = df[:-1]
    sentiment_score = get_sentiment_score(quote)
    train_lstm_model(df,sentiment_score)
    

@app.route('/sentiment-data', methods=['GET'])
def sentiment_data():
    symbol = request.args.get('symbol', 'AAPL').upper()
    days = int(request.args.get('days', 30))
    return jsonify(generate_sentiment_data(symbol, days))

@app.route('/search-stock', methods=['GET'])
def search_stock():
    query = request.args.get('query', '').upper()
    matches = [s for s in STOCK_SYMBOLS if query in s]
    return jsonify(matches)

@app.route('/')
def index():
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Stock & Sentiment Charts</title>
        <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
        <style>
            body { font-family: Arial, sans-serif; text-align: center; padding: 20px; background-color: #f8f9fa; }
            .controls { margin-bottom: 20px; position: relative; display: flex; justify-content: center; align-items: center; gap: 10px; }
            .buttons button { margin: 5px; padding: 10px 15px; font-size: 16px; cursor: pointer; border: none; background: #007bff; color: white; border-radius: 5px; transition: background 0.3s; }
            .buttons button:hover { background: #0056b3; }
            .buttons button.active { background: #28a745; }
            input { padding: 15px; font-size: 18px; width: 300px; border: 2px solid #007bff; border-radius: 5px; text-align: center; }
            .dropdown { position: absolute; top: 40px; width: 300px; background: white; border: 1px solid #ddd; display: none; z-index: 10; text-align: left; }
            .dropdown div { padding: 10px; cursor: pointer; }
            .dropdown div:hover { background: #f0f0f0; }
            .chart-container { display: flex; justify-content: center; gap: 30px; align-items: center; flex-wrap: wrap; margin-top: 20px; }
            .chart-box { width: 45%; max-width: 400px; background: white; padding: 15px; border-radius: 10px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1); height: 300px; display: flex; flex-direction: column; justify-content: center; align-items: center; }
            canvas { width: 100%; height: 100%; }
            .stock-card { background: white; padding: 20px; border-radius: 10px; display: inline-block; margin-top: 20px; box-shadow: 0px 4px 6px rgba(0, 0, 0, 0.1); font-size: 20px; }
            .stock-card .price { font-size: 30px; font-weight: bold; margin: 10px 0; }
            .stock-card .up { color: green; }
            .stock-card .down { color: red; }
            .search-button { padding: 15px 25px; font-size: 18px; border-radius: 5px; background-color: #007bff; color: white; border: none; cursor: pointer; transition: background-color 0.3s; }
            .search-button:hover { background-color: #0056b3; }
             /* Marquee Container */
        .news-marquee {
            width: 100%;
            overflow: hidden;
            background: #222;
            color: white;
            padding: 10px 0;
            position: relative;
            white-space: nowrap;
            font-family: Arial, sans-serif;
            margin-top: 20px;
        }


        /* Moving Text */
        .news-items {
            display: inline-block;
            white-space: nowrap;
            animation: scroll 20s linear infinite;
        }

        /* Marquee Animation */
        @keyframes scroll {
            from {
                transform: translateX(100%);
            }
            to {
                transform: translateX(-100%);
            }
        }

        /* Individual News Items */
        .news-items span {
            display: inline-block;
            padding: 0 30px;
            font-size: 18px;
            font-weight: bold;
        }

        .loader-overlay {
            position: fixed;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(255, 255, 255, 0.7); /* Transparent background */
            display: flex;
            justify-content: center;
            align-items: center;
            z-index: 1000;
            display: none;
        }

        .loader {
            border: 6px solid rgba(0, 0, 0, 0.1);
            border-top: 6px solid #3498db;
            border-radius: 50%;
            width: 50px;
            height: 50px;
            animation: spin 1s linear infinite;
        }

        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        </style>
    </head>
    <body>
        <h2>Stock Price & Sentiment Analysis - SentiNova</h2>
        
        <div class="controls">
            <input type="text" id="stockInput" value="AAPL" placeholder="Enter Stock Symbol" onkeyup="searchStock()">
            <button class="search-button" onclick="updateCharts()">Search</button>
            <div class="dropdown" id="stockDropdown"></div>
        </div>

<div id="loaderOverlay" class="loader-overlay">
        <div class="loader"></div>
    </div>
    <div class="buttons">
            <button id="btn30" onclick="updateCharts(30)">Last 30 Days</button>
            <button id="btn15" onclick="updateCharts(15)">Last 15 Days</button>
            <button id="btn7" onclick="updateCharts(7)">Last 7 Days</button>
        </div>
<div class="news-marquee">
        <div class="news-items">
            <span id="news1">🚨 Breaking: Stock Market Hits Record Highs</span>
            <span id="news2">🌍 World News: Global Summit Begins Tomorrow</span>
        </div>
    </div>
        <div class="stock-card" id="stockCard">
            <span>Today's Predicted Price:</span>
            <div class="price"><span id="predictedPrice">$0.00</span> <span id="priceIcon">📉</span></div>
            <div class="asOfDate" style="font-size: 12px;text-align: left;"><span id="date">As of 5th Feb 2025</span> </div>
        </div>

        <div class="chart-container">
            <div class="chart-box">
                <div>Stock Market Movement</div>
                <canvas id="stockChart"></canvas>
            </div>
            
            <div class="chart-box">
                <div>Sentiment Analysis</div>
                <canvas id="sentimentChart"></canvas>
            </div>
        </div>

        <script>
            let stockChart, sentimentChart;

            async function fetchStockData(symbol, days = 30) {
                const response = await fetch(`/stock-data?symbol=${symbol}&days=${days}`);
                const data = await response.json();
                return data;
            }

            async function fetchSentimentData(symbol, days = 30) {
                const response = await fetch(`/sentiment-data?symbol=${symbol}&days=${days}`);
                const data = await response.json();
                return data;
            }

            async function predictStockPrice(symbol) {
                const response = await fetch(`/predict-stock-value?symbol=${symbol}`);
                const data = await response.json();
                console.log("pavi",data);
                return data;
            }

            async function startBackgroundTraining(symbol) {
                fetch(`/start-background-training?symbol=${symbol}`);
            }

            async function updateCharts(days = 30) {
                showLoader()
                // Reset active button
                document.querySelectorAll('.buttons button').forEach(button => button.classList.remove('active'));
                document.getElementById(`btn${days}`).classList.add('active');

                const symbol = document.getElementById('stockInput').value.toUpperCase() || 'AAPL';
                const stockData = await fetchStockData(symbol, days);
                const sentimentData = await fetchSentimentData(symbol, days);
                
                console.log("pavi sentiment",sentimentData);
                

                if (stockData.error) {
                    alert('Error fetching stock data. Please check the symbol and try again.');
                    return;
                }

                // Update Stock Price Card
                console.log("pavi symbol", symbol)
                const temp = await predictStockPrice(symbol);
                console.log("pavi temp",temp);
                const predictedPrice = temp.value;
                const prevPrice = stockData.price.length > 1 ? stockData.price[stockData.price.length - 2] : predictedPrice;
                const priceChange = predictedPrice - prevPrice;
                document.getElementById('predictedPrice').innerText = `$${predictedPrice}`;
                document.getElementById('priceIcon').innerHTML = priceChange >= 0 ? "🔼" : "🔽";
                document.getElementById('priceIcon').className = priceChange >= 0 ? "up" : "down";

                document.getElementById('news1').innerText = `🚨 ${sentimentData.news[0]}`;
                document.getElementById('news2').innerText = `🌍 ${sentimentData.news[1]}`;
                //document.getElementById('news3').innerText = `💰 ${sentimentData.news[2]}`;

                var date = new Date();
                date.setDate(date.getDate() - 1);
                
                document.getElementById('date').innerText = date
                
                

                // Destroy existing charts to prevent overlapping data
                if (stockChart) stockChart.destroy();
                if (sentimentChart) sentimentChart.destroy();

                stockChart = new Chart(document.getElementById('stockChart').getContext('2d'), {
                    type: 'line',
                    data: {
                        labels: stockData.labels,
                        datasets: [{
                            label: 'Stock Price ($)',
                            data: stockData.price,
                            borderColor: 'blue',
                            backgroundColor: 'rgba(0,123,255,0.2)',
                            borderWidth: 2,
                            fill: true,
                            cubicInterpolationMode: 'monotone',  // Cubic Bezier curve
                            tension: 0.4  // Smooth curve
                        }]
                    },
                    options: {
                        responsive: true,
                        maintainAspectRatio: false,
                        scales: {
                            x: {
                                title: {
                                    display: true,
                                    text: 'Date'
                                },
                                grid: {
                                    display: false  // Remove background grid lines
                                }
                            },
                            y: {
                                title: {
                                    display: true,
                                    text: 'Price ($)'
                                },
                                grid: {
                                    display: false  // Remove background grid lines
                                }
                            }
                        }
                    }
                });

                // Update Sentiment Chart
                sentimentChart = new Chart(document.getElementById('sentimentChart').getContext('2d'), {
                    type: 'pie',
                    data: {
                        labels: sentimentData.labels,
                        datasets: [{
                            data: sentimentData.values,
                            backgroundColor: ['green', 'grey', 'red']
                        }]
                    },
                    options: {
                        responsive: true,
                        maintainAspectRatio: false
                    }
                });
                hideLoader()
                startBackgroundTraining(symbol)
            }

            async function searchStock() {
                const query = document.getElementById('stockInput').value;
                if (!query) return document.getElementById('stockDropdown').style.display = 'none';
                const response = await fetch(`/search-stock?query=${query}`);
                const stocks = await response.json();
                if (stocks.length === 0) {
                    document.getElementById('stockDropdown').style.display = 'none';
                } else {
                    document.getElementById('stockDropdown').innerHTML = stocks.map(s => `<div onclick="selectStock('${s}')">${s}</div>`).join('');
                    document.getElementById('stockDropdown').style.display = 'block';
                }
            }

            function selectStock(symbol) {
                document.getElementById('stockInput').value = symbol;
                document.getElementById('stockDropdown').style.display = 'none';
                updateCharts();
            }

            function showLoader() {
            document.getElementById("loaderOverlay").style.display = "flex";
        }

        function hideLoader() {
            document.getElementById("loaderOverlay").style.display = "none";
        }

            updateCharts();
        </script>
    </body>
    </html>
    """)

if __name__ == '__main__':
    app.run(debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
